In [1]:
#Saturated Oxygen Data Correction

In [49]:
from oxyfloat import *

print get_oxy_floats()

NameError: global name 'urllib2' is not defined

In [36]:
#This is to import libraries of different functions that allows the program do what it's doing
%pylab inline
from netCDF4 import Dataset
import pandas as pd
from pandas import Series , DataFrame
import csv
from IPython.core.display import Image
import urllib2
import StringIO
from thredds_crawler.crawl import Crawl
from seawater.extras import satO2
import time

Populating the interactive namespace from numpy and matplotlib


In [35]:
data = urllib2.urlopen("http://argo.jcommops.org/FTPRoot/Argo/Status/argo_all.txt").readlines()
data[1].replace('\0' , '').replace('\xff' , '').replace('\xfe' , '').replace('\r\n' , '')
#So this works for a specific row but not the whole thing O.o
#Make a loop that runs through all of the rows one by one (since all together doesn't work)

'6900877,99731,ARGOS,5058,5058,29/06/2014,17/04/2014,06,2014,57.5161,-21.8886,POSEIDON,P741,26/07/2015,59.7852,-22.6254,APEX,"Giese, Holger",holger.giese@bsh.de,Argo BSH,GERMANY,391.3461,0.86,1,79,0,0,0,120,1000,1200,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0'

In [3]:
d2 = [d.replace('\0' , '').replace('\xff' , '').replace('\xfe' , '').replace('\r\n' , '') for d in data]
d2[:5]

['WMO,TELECOM,TTYPE,MY_ID,SERIAL_NO,DATE0,NOTIF_DATE,MONTH0,YEAR0,LAT0,LON0,SHIP,CRUISE,DATE_,LAT_,LON_,MODEL,FULL_NAME,EMAIL,PROGRAM,COUNTRY,AGE,AGE_WEIGHT,GTS,PROFILES,DMPROFILES,GTSPRESS,GREYLIST,CYCLE_TIME,DRIFT_PRESS,PROFILE_PRESS,OXYGEN,FLUO,TURBIDITY,RAFOS,CDOM,BEAM,BACKSCATT,DOWNWELL,SST,SSS,PAL,OTHER,ICE_DETECT,ACTIVE,BEACHED,RETIRED',
 '3900380,27643,ARGOS," ",1510,09/02/2005,11/02/2005,02,2005,-56.47,-58.54," "," ",,0,0,APEX,"Turton, Jon",jon.turton@metoffice.gov.uk,Argo UK,UNITED KINGDOM,0,0,0,0,0,0,0,240,1500,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0',
 '3900381,27645,ARGOS," ",1511,09/02/2005,11/02/2005,02,2005,-58.78,-58.77," "," ",17/09/2009,-50.7143,33.9768,APEX,"Turton, Jon",jon.turton@metoffice.gov.uk,Argo UK,UNITED KINGDOM,1681.1643,0,1,168,168,2003,0,240,1500,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0',
 '3900382,27652,ARGOS," ",1512,09/02/2005,11/02/2005,02,2005,-57.49,-58.9," "," ",,0,0,APEX,"Turton, Jon",jon.turton@metoffice.gov.uk,Argo UK,UNITED KINGDOM,0,0,0,0,0,0,0,240,

In [4]:
#Latest attempt
df = pd.DataFrame(d2)
type(df)

pandas.core.frame.DataFrame

In [5]:
with open('d2.csv', 'w') as f:
    for row in d2:
        f.write(row + '\n')

df = pd.read_csv('d2.csv')
df[:5]

,WMO,TELECOM,TTYPE,MY_ID,SERIAL_NO,DATE0,NOTIF_DATE,MONTH0,YEAR0,LAT0,...,BACKSCATT,DOWNWELL,SST,SSS,PAL,OTHER,ICE_DETECT,ACTIVE,BEACHED,RETIRED
0,3900380,27643,ARGOS,,1510,09/02/2005,11/02/2005,2,2005,-56.4700,...,0,0,0,0,0,0,0,0,0,0
1,3900381,27645,ARGOS,,1511,09/02/2005,11/02/2005,2,2005,-58.7800,...,0,0,0,0,0,0,0,0,0,0
2,3900382,27652,ARGOS,,1512,09/02/2005,11/02/2005,2,2005,-57.4900,...,0,0,0,0,0,0,0,0,0,0
3,3900399,46233,ARGOS,1201,,07/02/2005,14/02/2005,2,2005,-25.2461,...,0,0,0,0,0,0,0,0,0,0
4,3900400,46234,ARGOS,1202,,09/02/2005,14/02/2005,2,2005,-22.7369,...,0,0,0,0,0,0,0,0,0,0


In [6]:
#This prints out certain columns (of my choosing) with all the rows still present
fd_oga = df.loc[: , ['WMO' , 'OXYGEN' , 'GREYLIST' , 'AGE' , 'DATE_' , 'LAT_' , 'LON_'] ] # oga = oxygen, grey list, and age
fd_oga[:5]

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
0,3900380,0,0,0.0000,NaN,0.0000,0.0000
1,3900381,0,0,1681.1643,17/09/2009,-50.7143,33.9768
2,3900382,0,0,0.0000,NaN,0.0000,0.0000
3,3900399,0,1,2001.7359,01/08/2010,-26.0516,-42.4516
4,3900400,0,1,1941.0619,04/06/2010,-26.5448,-42.5100


In [7]:
#This takes the fd_oga table and only shows the rows that have oxygen data
fd_oxygen = fd_oga.loc[ fd_oga.loc[: , 'OXYGEN'] == 1 , :]
fd_oxygen[:5]

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
84,5900245,1,0,1097.1958,05/01/2006,-36.2894,165.6669
196,4900637,1,0,1640.5049,21/08/2009,45.9560,-128.0490
317,4900497,1,0,1030.1457,26/02/2007,34.7538,-63.6900
368,2900460,1,0,792.1306,26/04/2007,44.4140,-153.7139
436,4900651,1,0,1200.1958,12/11/2008,38.4279,-157.9447


In [8]:
#This takes the fd_oga table and only shows the rows that aren't grey listed     #gl = grey list
fd_gl = fd_oga.loc[ fd_oga.loc[: , 'GREYLIST'] == 0 , :] #gl = grey list
fd_gl[:5]

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
0,3900380,0,0,0.0000,NaN,0.0000,0.0000
1,3900381,0,0,1681.1643,17/09/2009,-50.7143,33.9768
2,3900382,0,0,0.0000,NaN,0.0000,0.0000
5,5900339,0,0,2781.0015,23/09/2012,-56.0493,-65.4248
6,5900346,0,0,3130.1958,08/09/2013,-58.1230,-22.3040


In [9]:
#This takes the fd_oga table and only shows the rows that are at least 340 days old
fd_age = fd_oga.loc[ fd_oga.loc[: , 'AGE'] >= 340. , :]
fd_age[:5]

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
1,3900381,0,0,1681.1643,17/09/2009,-50.7143,33.9768
3,3900399,0,1,2001.7359,01/08/2010,-26.0516,-42.4516
4,3900400,0,1,1941.0619,04/06/2010,-26.5448,-42.5100
5,5900339,0,0,2781.0015,23/09/2012,-56.0493,-65.4248
6,5900346,0,0,3130.1958,08/09/2013,-58.1230,-22.3040


In [10]:
#Merging the fd_oxygen table with the fd_gl table
fd_merge_og = pd.merge(fd_oxygen , fd_gl) #og = oxygen and grey list
fd_merge_og[:5]

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
0,5900245,1,0,1097.1958,05/01/2006,-36.2894,165.6669
1,4900637,1,0,1640.5049,21/08/2009,45.9560,-128.0490
2,4900497,1,0,1030.1457,26/02/2007,34.7538,-63.6900
3,2900460,1,0,792.1306,26/04/2007,44.4140,-153.7139
4,4900651,1,0,1200.1958,12/11/2008,38.4279,-157.9447


In [11]:
#Later work will go off of this table

#FINALLY HAVE ONLY THE DATA I DESIRE
#PRAISE HELIX
#After all the trial and error (=^.^=)

fd_merge_oga = pd.merge(fd_merge_og , fd_age)
fd_merge_oga[:5]
#Merged the fd_merge_og table with the fd_age table

#Inactive -> active = 0

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
0,5900245,1,0,1097.1958,05/01/2006,-36.2894,165.6669
1,4900637,1,0,1640.5049,21/08/2009,45.9560,-128.0490
2,4900497,1,0,1030.1457,26/02/2007,34.7538,-63.6900
3,2900460,1,0,792.1306,26/04/2007,44.4140,-153.7139
4,4900651,1,0,1200.1958,12/11/2008,38.4279,-157.9447


In [12]:
gd = urllib2.urlopen("ftp://ftp.ifremer.fr/ifremer/argo/ar_index_global_meta.txt").readlines()
gd[:10]

['# Title : Metadata directory file of the Argo Global Data Assembly Center\n',
 '# Description : The directory file describes all metadata files of the argo GDAC ftp site.\n',
 '# Project : ARGO\n',
 '# Format version : 2.0\n',
 '# Date of update : 20150727162538\n',
 '# FTP root number 1 : ftp://ftp.ifremer.fr/ifremer/argo/dac\n',
 '# FTP root number 2 : ftp://usgodae.usgodae.org/pub/outgoing/argo/dac\n',
 '# GDAC node : CORIOLIS\n',
 'file,profiler_type,institution,date_update\n',
 'aoml/13857/13857_meta.nc,845,AO,20120521144513\n']

In [43]:
#May or may not even need this
gd = urllib2.urlopen("ftp://ftp.ifremer.fr/ifremer/argo/ar_index_global_meta.txt").readlines() #gd = global data

##gd1 = [g.replace('\0' , '').replace('\xff' , '').replace('\xfe' , '').replace('\r\n' , '') for g in gd]

gd1[:10]
#Now to get it so when a user inputs a float number, the DAC of that number shows up

['# Title : Metadata directory file of the Argo Global Data Assembly Center\n',
 '# Description : The directory file describes all metadata files of the argo GDAC ftp site.\n',
 '# Project : ARGO\n',
 '# Format version : 2.0\n',
 '# Date of update : 20150727162538\n',
 '# FTP root number 1 : ftp://ftp.ifremer.fr/ifremer/argo/dac\n',
 '# FTP root number 2 : ftp://usgodae.usgodae.org/pub/outgoing/argo/dac\n',
 '# GDAC node : CORIOLIS\n',
 'file,profiler_type,institution,date_update\n',
 'aoml/13857/13857_meta.nc,845,AO,20120521144513\n']

In [14]:
with open('gd1.csv', 'w') as f:
    for row in gd1:
        f.write(row + '\n')

gd_table = pd.read_csv('gd1.csv' , comment = '#')
gd_table[:5]

,file,profiler_type,institution,date_update
0,aoml/13857/13857_meta.nc,845,AO,20120521144513
1,aoml/13858/13858_meta.nc,845,AO,20130222100319
2,aoml/13859/13859_meta.nc,845,AO,20120521144513
3,aoml/15819/15819_meta.nc,845,AO,20120521144513
4,aoml/15820/15820_meta.nc,845,AO,20120521144513


In [15]:
# Would like to read off Internet
gd_table.loc[:,['file']][:5]

,file
0,aoml/13857/13857_meta.nc
1,aoml/13858/13858_meta.nc
2,aoml/13859/13859_meta.nc
3,aoml/15819/15819_meta.nc
4,aoml/15820/15820_meta.nc


In [16]:
good_floats = []
for index,row in fd_merge_oga.loc[:,['WMO']].iterrows():
    good_floats.append(row['WMO'])
print good_floats[:10]

['5900245', '4900637', '4900497', '2900460', '4900651', '4900652', '5900420', '5900421', '5900422', '3900274']


In [17]:
dac_substring = []
all_floats = []
for index,row in gd_table.loc[:,['file']].iterrows():
    floatNum = row['file'].split('/')[1]
    all_floats.append(floatNum)

    if floatNum in good_floats:
        dac_substring.append(row['file'])
    

In [18]:
i=0
dacUrls = []
for dac in dac_substring:
    tdsUrl = ''
    tdsUrl += "http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/"
    tdsUrl += '/'.join(dac.split('/')[:2])
    tdsUrl += "/profiles/catalog.xml"
    dacUrls.append(tdsUrl)
    i += 1

In [19]:
print len(dacUrls)
dacUrls[:5]

553


['http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/aoml/1900722/profiles/catalog.xml',
 'http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/aoml/1901378/profiles/catalog.xml',
 'http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/aoml/2900114/profiles/catalog.xml',
 'http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/aoml/2900115/profiles/catalog.xml',
 'http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/aoml/2900116/profiles/catalog.xml']

In [20]:
for dac_url in dacUrls:
    start_time = time.time()
    print "Crawling " + dac_url
    sys.stdout.flush()
    c = Crawl(dac_url)
    urls = [s.get("url") for d in c.datasets 
                for s in d.services if s.get("service").lower() == "opendap"]
    sec_diff = time.time() - start_time
    print "Found %s netCDF files in %.1f seconds" % (len(urls), sec_diff) 

Crawling http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/aoml/1900722/profiles/catalog.xml
Found 405 netCDF files in 168.2 seconds
Crawling http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/aoml/1901378/profiles/catalog.xml
Found 265 netCDF files in 106.5 seconds
Crawling http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/aoml/2900114/profiles/catalog.xml
Found 216 netCDF files in 84.9 seconds
Crawling http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/aoml/2900115/profiles/catalog.xml
Found 118 netCDF files in 45.4 seconds
Crawling http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/aoml/2900116/profiles/catalog.xml
Found 239 netCDF files in 94.3 seconds
Crawling http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/aoml/2900117/profiles/catalog.xml
Found 236 netCDF files in 93.1 seconds
Crawling http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/aoml/2900118/profiles/catalog.xml
Found 130 netCDF files in 50.0 seconds
Crawling http://thredds.a

ConnectionError: ('Connection aborted.', error(104, 'Connection reset by peer'))

In [ ]:
def conv

In [ ]:
floatLookup = {}
for url in urls:
    print url
    ds = Dataset(url)
    floatNum = url.split('/')[9]
    print floatNum
    p,t,s,o = (ds.variables['PRES_ADJUSTED'][0][0], ds.variables['TEMP_ADJUSTED'][0][0], 
               ds.variables['PSAL_ADJUSTED'][0][0], ds.variables['DOXY_ADJUSTED'][0][0])
    lat,lon,mtime = (ds.variables['LATITUDE'][0], ds.variables['LONGITUDE'][0], ds.variables['JULD'][0])
    
    # Convert o from micromol/kg to ml/l 
    ##perc_sat = 100 * convertTomll(o) / satO2(s,t) 
    print s,t,perc_sat
    floatLookup[floatNum] = (p,t,s,o,lat,lon,mtime)

In [ ]:
floatLookup

In [33]:
p,t,s,o = (ds.variables['PRES_ADJUSTED'][0][0], ds.variables['TEMP_ADJUSTED'][0][0], ds.variables['PSAL_ADJUSTED'][0][0], ds.variables['DOXY_ADJUSTED'][0][0])

In [53]:
p,t,s,o

(4.3000002, 29.790001, 34.827457, 206.49001)

In [35]:
lat,lon,mtime = (ds.variables['LATITUDE'][0], ds.variables['LONGITUDE'][0], ds.variables['JULD'][0])

In [36]:
lat,lon,mtime

(6.0310000000000006, -14.026, 21985.069212962964)

In [118]:
#The p here is for depth...maybe
#That's why it said the unit was km
def dens_formula(p , t , s):
    C = (999.83 + (5.053*p) - (.048*(p**2))
    B =  .808 - (.0085*p)
    a = .0708*((1 + (.351*p)) + .068*(1 - (.0683*p))*t)
    y = .003*(1 - (.059*p) - .012*(1 - .064*p)*t)
    dens = C + (B*s) - (a*t) - ((y*(35 - s))*t)
    return dens

SyntaxError: invalid syntax (<ipython-input-118-819d3bb21dd0>, line 5)

In [115]:
dens_formula(p , t , s)

1039.220347679988

In [ ]:
def dens_formula(p , t , s):
    p = p / 10.0
    pw = 999.842594 + ((6.793952 * (10**-abs(2)))*t) - ((9.095290 * (10**-abs(3)))*(t**2)) + ((1.001685 * (10**-abs(4)))*(t**3)) - ((1.120083 * (10**-abs(6)))*(t**4)) + ((6.536332 * (10**-abs(9))*(t**5))        
    